# Calculate wind speed (hourly)

In [1]:
from dask.distributed import Client,LocalCluster
from dask_jobqueue import PBSCluster

In [18]:
# One node on Gadi has 48 cores - try and use up a full node before going to multiple nodes (jobs)

walltime = "00:30:00"
cores = 48
memory = str(4 * cores) + "GB"

cluster = PBSCluster(walltime=str(walltime), cores=cores, memory=str(memory), processes=cores,
                     job_extra_directives=["-q normal",
                                           "-P dt6",
                                           "-l ncpus="+str(cores),
                                           "-l mem="+str(memory),
                                           "-l storage=gdata/xp65+gdata/w42+gdata/rt52"],
                     local_directory="$TMPDIR",
                     job_directives_skip=["select"])

In [19]:
cluster.scale(jobs=1)
client = Client(cluster)

In [20]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.121.4:41253,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [5]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt

In [7]:
%cd /g/data/w42/dr6273/work/power_models

import functions as fn

/g/data/w42/dr6273/work/power_models


In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
root_path = "/g/data/rt52/era5/single-levels/reanalysis/"
write_path = "/g/data/w42/dr6273/work/data/era5/100w/hourly/"

In [10]:
def get_Aus_boundary():
    """
    Returns a list of bounding coordinates for Australia.
    """
    return [110, 155, -10, -45]

# Compute wind speed by year

In [11]:
def windspeed(u, v):
    """
    Compute windspeed from u and v
    
    u: array of zonal wind
    v: array of meridional wind
    """
    return np.sqrt(u ** 2 + v ** 2)

In [12]:
def preprocess(ds):
    """
    Preprocess function for open_mfdataset.
    Selects Australian region and renames coords.
    """
    ds = ds.sel(
        longitude=slice(region[0], region[1]),
        latitude=slice(region[2], region[3])
    )
    ds = ds.rename(
        {'longitude': 'lon',
         'latitude': 'lat'}
    )
    ds = ds.chunk({"time": -1, "lat": -1, "lon": -1})
    return ds

In [13]:
def load_hourly(preprocess, variable, year, first_hour, data_path=root_path):
    """
    Load and preprocess hourly data for a given year
    
    preprocess: preprocess function
    variable: name of variable to process
    year: year to process
    first_hour: desired first hour from which to compute 24-hour aggregations
    data_path: path to hourly data
    """
    # Open all hours in the year (~33 GB)
    hourly = xr.open_mfdataset(
        data_path + variable + "/" + str(year) + "/*.nc",
        preprocess=preprocess
    )

    # Start the aggregation on the desired hour (e.g. 0000)
    data_first_hour = hourly["time"].dt.hour.item(0)
    desired_start_index = (first_hour - data_first_hour) % 24
    hourly = hourly.isel(time=range(desired_start_index, len(hourly["time"])))
    
    return hourly

In [14]:
region = get_Aus_boundary()

In [15]:
region

[110, 155, -10, -45]

In [16]:
years = range(2024, 2025)

range(2024, 2025)

In [21]:
for year in years:
    if year % 5 == 0:
        print(year)
        
    u100 = load_hourly(
        preprocess,
        "100u",
        year,
        0
    )

    v100 = load_hourly(
        preprocess,
        "100v",
        year,
        0
    )
    
    w100 = windspeed(
        u100.rename({"u100": "w100"}),
        v100.rename({"v100": "w100"})
    )
    w100 = w100.chunk({"time": 24*7*12})
    
    encoding = {
        "w100": {"dtype": "float32"}
    }
    w100.to_netcdf(
        write_path + "100w_era5_hourly_" + str(year) + "_Aus.nc",
        mode="w",
        encoding=encoding
    )

# Close cluster

In [22]:
client.close()
cluster.close()

INFO:dask_jobqueue.pbs:Resource specification for PBS not set, initializing it to select=1:ncpus=48:mem=179GB
